In [ ]:
%load_ext autoreload
%autoreload 2

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from space_env import Satellite, Propagator
from visualizer import plot_ground_track, graficar_2d_plotly, plot_ground_track_with_access
import constants as cts
from datetime import time
from astropy.time import Time

In [4]:
# Mission Settings
# LEO orbit circular
h = 500 # km
LTAN = time(9,0,0)
Lon_RAAN= LTAN.hour*15 # [deg/hs]


In [5]:
# Instanciamos el satelite (esto crea el objeto Spacecraft en GMAT)
sat = Satellite("Salta")

sat.set_keplerian(
    sma=cts.Re + h, 
    ecc=0.0001, 
    inc=98.2, 
    raan=Lon_RAAN, 
    aop=0, 
    ta=0
)

print(f"Satelite '{sat.name}' configurado.")

Satelite 'Salta' configurado.


In [6]:
# 1. Definir la fidelidad de la física
config_leo = {
    'gravity': 'Earth',
    'degree': 2,    # Activa J2
    'order': 0,
    'drag': True    # Resistencia atmosférica (importante en LEO)
}

# 2. Instanciar el propagador (esto crea el ForceModel y el Propagator en GMAT)
prop = Propagator("MyPropagator", config=config_leo)


GravityField  

   Field                                   Type   Value
   --------------------------------------------------------

   Degree                               Integer   4
   Order                                Integer   4
   StmLimit                             Integer   100
   PotentialFile                       Filename   JGM2.cof
   TideFile                            Filename   
   TideModel                             String   None



In [7]:
# 5 hours 
duracion_sim = 5 * 3600  # [sec]

print("Propagating ...")
# Esto devuelve un array de numpy: [tiempo, x, y, z]
trajectory = prop.run(sat, duration_sec=duracion_sim, step_size=60)

print(f" Ready -  {trajectory.shape[0]} data points available")

Propagating ...
 Ready -  300 data points available


In [13]:
# Ground Track
figura = plot_ground_track(trajectory, epoch="2026-02-10T12:00:00")
figura.show()

In [ ]:
# Ejecución
fig_2d = graficar_2d_plotly(trajectory, epoch="2026-02-10T12:00:00")
fig_2d.show()

# Access Analysis

In [16]:
import ground_segment as gs

In [17]:
salta_site = gs.Site("Salta",-24.7851,-65.39939, 1200)
# bsas = gs.Site("BuenosAires", -34.6, -58.48, 25)
# matera = gs.Site("Matera", 40.66, 16.6, 100)

In [18]:
import access_manager as am

In [22]:
trajectory[2]

array([  120.        , -4629.34523384,  4903.94275179,  1347.41363477])

In [25]:
acc1 = am.AccessManager()
astropy_time_start = Time("2026-02-10T12:00:00", format='isot', scale='utc')
access_salta= acc1.calculate_access(trajectory, astropy_time_start, salta_site)


In [28]:
visual_data = [{'site': salta_site, 'passes': access_salta}]
fig = plot_ground_track_with_access(trajectory, astropy_time_start, visual_data)
fig.show()